# Evaluate solar MED model using experimental data as system inputs

And save the results in a csv file to be visualized in `visualize_test.ipynb`

In [13]:
# Setup environment for running MATLAB code if not done externally
import os

os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"

from models_psa import solar_MED

In [14]:
from pathlib import Path
import os
import hjson
import numpy as np
import pandas as pd
from IPython.display import display
from loguru import logger
import time
from phd_visualizations.utils import rename_signal_ids_to_var_ids
from phd_visualizations.utils.units import unit_conversion


# auto reload modules
%load_ext autoreload

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'

# Debería ser un .csv al que se le hayan añadido las variables faltantes desde librescada:
# - J de variadores y medidor de potencia
# - FT-DES-002_VFD

filename_process_data = '20231030_solarMED.csv'
filename_process_data2 = '20231030_MED.csv'

# Resample figures using plotly_resampler
resample_figures = False

sample_rate = '60s'
sample_rate_numeric = int(sample_rate[:-1])

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Load variables information
with open( Path("data") / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)
    
# Read data from csv, the index column is the one named "time", which is not the first one
df = pd.read_csv(data_path / filename_process_data, parse_dates=True, index_col='TimeStamp')
# Rename index column to "time"
df.index.names = ['time']
# Set UTC timezone
df = df.tz_localize('UTC')

# TEMP: Read additional MED data from librescada logging, in the future everything should be read directly from librescada
df_aux = pd.read_csv(data_path / filename_process_data2, parse_dates=True, index_col='time')
# Set UTC timezone
df_aux = df_aux.tz_localize('UTC')
# Add columns from df_aux to df
df = pd.concat([df, df_aux], axis=1)

# Preprocessing
# Sample every `sample_rate` seconds to reduce the size of the dataframe
df = df.resample(sample_rate).mean()

# Rename columns from signal_id to var_id
df = rename_signal_ids_to_var_ids(df, vars_config)

# Convert units to model units
df = unit_conversion(df, vars_config, input_unit_key='units_scada', output_unit_key='units_model')

display(df.head())

2024-02-12 14:29:42.721 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tamb to C from C
2024-02-12 14:29:42.722 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_in to C from C
2024-02-12 14:29:42.723 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_in to C from C
2024-02-12 14:29:42.724 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_out to C from C
2024-02-12 14:29:42.724 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_out to C from C
2024-02-12 14:29:42.725 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tsf_in to C from C
2024-02-12 14:29:42.726 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tsf_out to C from C
2024-02-12 14:29:42.727 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tts_h_t_in to C from C
2024-02-12 14:29:42.727 | DEBUG    | phd_visual

,qmed_s,qts_dis,Tts_h_out,Tmed_s_in,Tts_c_b_in,wmed_f,qmed_f,qmed_d,Tmed_c_out,Tmed_c_in,...,Thx_p_out,Thx_s_in,Thx_s_out,qhx_p,qhx_s,T3wv_src,T3wv_dis_in,T3wv_dis_out,q3wv_src,q3wv_dis
time,,,,,,,,,,,,,,,,,,,,,
2023-10-30 07:30:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.03125
2023-10-30 07:31:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.03125
2023-10-30 07:32:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.03125
2023-10-30 07:33:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.03125
2023-10-30 07:34:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.03125


In [16]:
logger.debug(list(df.columns))

2024-02-12 14:29:42.785 | DEBUG    | __main__:<module>:1 - ['qmed_s', 'qts_dis', 'Tts_h_out', 'Tmed_s_in', 'Tts_c_b_in', 'wmed_f', 'qmed_f', 'qmed_d', 'Tmed_c_out', 'Tmed_c_in', 'Tamb', 'qts_src', 'qsf', 'I', 'Tts_c_t', 'Tts_c_m', 'Tts_c_b', 'Tts_h_t', 'Tts_h_m', 'Tts_h_b', 'Tts_c_b_out', 'Tts_h_t_in', 'Tsf_in', 'Tsf_out', 'Jmed_d', 'Jmed_s_f', 'Jmed_b', 'Tmed_s_out', 'Jmed_c', 'qmed_c', 'Tts_src_in', 'Tts_dis', 'Tts_dis_in', 'Thx_p_in', 'Thx_p_out', 'Thx_s_in', 'Thx_s_out', 'qhx_p', 'qhx_s', 'T3wv_src', 'T3wv_dis_in', 'T3wv_dis_out', 'q3wv_src', 'q3wv_dis']


In [17]:
# Filter out nans until first value in Tts

start_idx = df.index.get_loc( df['Tts_c_b'].first_valid_index() )

df = df.iloc[start_idx:]

logger.info(f'Removed up to row {start_idx} to filter NaN values in thermal storage state')

2024-02-12 14:29:44.071 | INFO     | __main__:<module>:7 - Removed up to row 14 to filter NaN values in thermal storage state


In [18]:
from models_psa import solar_MED

%autoreload 2

# Initialize combined system model
model = solar_MED(
    ts=sample_rate_numeric,
    curve_fits_path='data/curve_fits.json',
    
    # Initial states. Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[0], df['Tts_h_m'].iloc[0], df['Tts_h_b'].iloc[0]], 
    Tts_c=[df['Tts_c_t'].iloc[0], df['Tts_c_m'].iloc[0], df['Tts_c_b'].iloc[0]],
    cost_w = cost_w, # €/m³ 
    cost_e = cost_e # €/kWhe
)

2024-02-12 14:29:55.629 | INFO     | models_psa:__post_init__:296 - MATLAB MED model initialized
2024-02-12 14:29:55.633 | DEBUG    | models_psa:__post_init__:302 - Curve fits file loaded from data/curve_fits.json
2024-02-12 14:29:55.633 | DEBUG    | models_psa:__post_init__:307 - Initialization completed


In [19]:
# Run simulation
L = len(df)

results = []
# for idx in range(1, L):
# Iterate over the dataframe
for idx, (time, row) in enumerate(df.iterrows()):
    start_time = time.time()
    
    model.step(
        # Decision variables
        ## MED
        Tmed_s_in=row["Tmed_s_in"],
        Tmed_c_out=row["Tmed_c_out"],
        mmed_s=row["qmed_s"],
        mmed_f=row["qmed_f"],
        
        ## Thermal storage
        mts_src=row["qts_src"],
        
        ## Solar field
        Tsf_out=row["Tsf_out"],
        
        # Environment variables
        Tamb=row["Tamb"],
        I=row["I"],
        Tmed_c_in=row["Tmed_c_in"],
        wmed_f=35.5, # g/kg
    )
    
    # cost = model.calculate_cost()
    
    print(f'Iteration {idx} of {L} completed, took {time.time() - start_time:.3f} seconds, test time: {time:}')
    
    # Store system states
    results.append( model.get_properties() )

# Terminate 
model.terminate()

ValueError: Value of mmed_s must be a number within: [20.016, 53.28] (0.031250016000000005)